# DATASCI 315, Group Work 10: Transformers

In this assignment, you will explore the core components of transformer architectures, which power modern language models like GPT and BERT. You will implement attention mechanisms, understand positional encodings, and build a simple transformer-based model.

**Learning Objectives:**
- Understand scaled dot-product attention
- Implement causal (autoregressive) masking for language modeling
- Work with positional encodings
- Use PyTorch's transformer modules

**Resources:**
- [Attention Is All You Need (original paper)](https://arxiv.org/abs/1706.03762)
- [PyTorch Transformer Documentation](https://pytorch.org/docs/stable/nn.html#transformer-layers)
- [The Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/)

## Setup

First, let's import the necessary libraries.

In [ ]:
import math

import torch
from torch import nn
from torch.nn import functional

## Background: Scaled Dot-Product Attention

The core operation in transformers is **scaled dot-product attention**. Given queries $Q$, keys $K$, and values $V$, attention is computed as:

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

where $d_k$ is the dimension of the keys (used for scaling to prevent the dot products from growing too large).

**Why scale by $\sqrt{d_k}$?** When the dimension $d_k$ is large, the dot products $QK^T$ can have large magnitudes, pushing the softmax into regions with very small gradients. Scaling by $\sqrt{d_k}$ keeps the variance of the dot products roughly constant regardless of dimension.

**Masking:** For autoregressive (causal) language modeling, we need to prevent the model from "seeing the future." This is done by adding a mask that sets future positions to $-\infty$ before the softmax, so they contribute zero weight.

### Problem 1: Implement Scaled Dot-Product Attention

Implement the `scaled_dot_product_attention` function that computes attention scores and applies them to the values.

**Parameters:**
- `query`: Tensor of shape `(batch_size, seq_len, embed_dim)`
- `key`: Tensor of shape `(batch_size, seq_len, embed_dim)`
- `value`: Tensor of shape `(batch_size, seq_len, embed_dim)`
- `mask`: Optional boolean tensor of shape `(seq_len, seq_len)` where `True` indicates positions to mask (set to $-\infty$)

**Returns:**
- `output`: Tensor of shape `(batch_size, seq_len, embed_dim)` - the attention-weighted values
- `attention_weights`: Tensor of shape `(batch_size, seq_len, seq_len)` - the attention weights after softmax

**Worked Example:**

Consider a simple case with `batch_size=1`, `seq_len=2`, `embed_dim=2`:

```
Q = [[1, 0],    K = [[1, 0],    V = [[1, 2],
     [0, 1]]         [0, 1]]         [3, 4]]
```

Step 1: Compute $QK^T$:
```
QK^T = [[1*1 + 0*0, 1*0 + 0*1],   = [[1, 0],
        [0*1 + 1*0, 0*0 + 1*1]]      [0, 1]]
```

Step 2: Scale by $\sqrt{d_k} = \sqrt{2} \approx 1.414$:
```
Scaled = [[0.707, 0],
          [0, 0.707]]
```

Step 3: Apply softmax row-wise:
```
Attention weights = [[0.67, 0.33],
                     [0.33, 0.67]]
```

Step 4: Multiply by V:
```
Output = [[0.67*1 + 0.33*3, 0.67*2 + 0.33*4],
          [0.33*1 + 0.67*3, 0.33*2 + 0.67*4]]
       = [[1.67, 2.67],
          [2.33, 3.33]]
```

**Hint:** Use `torch.bmm` for batched matrix multiplication and `F.softmax` with `dim=-1`.

In [ ]:
def scaled_dot_product_attention(query, key, value, mask=None):
    """
    Compute scaled dot-product attention.

    Parameters:
    - query: Tensor of shape (batch_size, seq_len, embed_dim)
    - key: Tensor of shape (batch_size, seq_len, embed_dim)
    - value: Tensor of shape (batch_size, seq_len, embed_dim)
    - mask: Optional boolean tensor of shape (seq_len, seq_len) where True = mask out

    Returns:
    - output: Tensor of shape (batch_size, seq_len, embed_dim)
    - attention_weights: Tensor of shape (batch_size, seq_len, seq_len)
    """
    # BEGIN SOLUTION
    # Get the embedding dimension for scaling
    embed_dim = query.size(-1)
    scale = math.sqrt(embed_dim)

    # Compute attention scores: Q @ K^T / sqrt(d_k)
    # query: (batch, seq_len, embed_dim)
    # key.transpose(-2, -1): (batch, embed_dim, seq_len)
    # scores: (batch, seq_len, seq_len)
    scores = torch.bmm(query, key.transpose(-2, -1)) / scale

    # Apply mask if provided (set masked positions to -inf before softmax)
    if mask is not None:
        scores = scores.masked_fill(mask, float("-inf"))

    # Apply softmax to get attention weights
    attention_weights = functional.softmax(scores, dim=-1)

    # Apply attention weights to values
    output = torch.bmm(attention_weights, value)

    return output, attention_weights
    # END SOLUTION

In [ ]:
# Test assertions
# Test basic attention computation
torch.manual_seed(42)
batch_size, seq_len, embed_dim = 2, 4, 8
query = torch.randn(batch_size, seq_len, embed_dim)
key = torch.randn(batch_size, seq_len, embed_dim)
value = torch.randn(batch_size, seq_len, embed_dim)

output, attention_weights = scaled_dot_product_attention(query, key, value)

assert output.shape == (
    batch_size,
    seq_len,
    embed_dim,
), f"Output shape should be {(batch_size, seq_len, embed_dim)}, got {output.shape}"
expected_attn_shape = (batch_size, seq_len, seq_len)
assert (
    attention_weights.shape == expected_attn_shape
), f"Attention weights shape should be {expected_attn_shape}, got {attention_weights.shape}"

# Attention weights should sum to 1 along the last dimension
weight_sums = attention_weights.sum(dim=-1)
assert torch.allclose(
    weight_sums, torch.ones_like(weight_sums), atol=1e-5
), "Attention weights should sum to 1 along last dimension"
print("All tests passed!")

# BEGIN HIDDEN TESTS
# Test with identity-like inputs
query_id = torch.eye(3).unsqueeze(0)  # (1, 3, 3)
key_id = torch.eye(3).unsqueeze(0)
value_id = torch.arange(9).float().reshape(1, 3, 3)
out_id, weights_id = scaled_dot_product_attention(query_id, key_id, value_id)
# With identity Q and K, attention should focus more on diagonal
assert weights_id[0, 0, 0] > weights_id[0, 0, 1], "Diagonal attention should be higher"

# Test that masking works
mask = torch.ones(4, 4, dtype=torch.bool)
mask[0, 0] = False  # Only allow position 0 to attend to position 0
_, masked_weights = scaled_dot_product_attention(query, key, value, mask=mask)
assert torch.allclose(
    masked_weights[:, 0, 0], torch.ones(batch_size), atol=1e-5
), "With full mask except [0,0], attention should be 1.0 at [0,0]"
# END HIDDEN TESTS

### Problem 2: Create a Causal Mask

For autoregressive language models (like GPT), each position should only attend to previous positions (including itself). This requires a **causal mask** (also called a "look-ahead mask").

Implement `create_causal_mask` that returns a boolean mask where `True` indicates positions that should be masked out (i.e., future positions).

**Parameters:**
- `seq_len`: The sequence length

**Returns:**
- A boolean tensor of shape `(seq_len, seq_len)` where `mask[i, j] = True` if position `i` should NOT attend to position `j` (i.e., if `j > i`)

**Example for seq_len=4:**
```
[[False,  True,  True,  True],   # Position 0 can only see position 0
 [False, False,  True,  True],   # Position 1 can see positions 0, 1
 [False, False, False,  True],   # Position 2 can see positions 0, 1, 2
 [False, False, False, False]]   # Position 3 can see all positions
```

**Hint:** Use `torch.triu` (upper triangular) with `diagonal=1` to create a matrix of ones above the diagonal, then convert to boolean.

In [ ]:
def create_causal_mask(seq_len):
    """
    Create a causal mask for autoregressive attention.

    Parameters:
    - seq_len: Length of the sequence

    Returns:
    - mask: Boolean tensor of shape (seq_len, seq_len) where True = mask out
    """
    # BEGIN SOLUTION
    # Create upper triangular matrix with ones above the diagonal
    # This masks out future positions (j > i)
    return torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool()
    # END SOLUTION

In [ ]:
# Test assertions
mask = create_causal_mask(4)

assert mask.shape == (4, 4), f"Mask shape should be (4, 4), got {mask.shape}"
assert mask.dtype == torch.bool, f"Mask should be boolean, got {mask.dtype}"

# Check specific values
expected = torch.tensor(
    [
        [False, True, True, True],
        [False, False, True, True],
        [False, False, False, True],
        [False, False, False, False],
    ]
)
assert torch.equal(mask, expected), f"Mask values incorrect.\nExpected:\n{expected}\nGot:\n{mask}"
print("All tests passed!")

# BEGIN HIDDEN TESTS
# Test with different sizes
mask_2 = create_causal_mask(2)
expected_2 = torch.tensor([[False, True], [False, False]])
assert torch.equal(mask_2, expected_2), "Causal mask for seq_len=2 is incorrect"

mask_1 = create_causal_mask(1)
expected_1 = torch.tensor([[False]])
assert torch.equal(mask_1, expected_1), "Causal mask for seq_len=1 is incorrect"

# Verify diagonal is always False (can attend to self)
mask_5 = create_causal_mask(5)
assert not mask_5.diagonal().any(), "Diagonal should be all False (can attend to self)"
# END HIDDEN TESTS

### Problem 3: Verify Causal Masking Works

Now let's verify that the causal mask properly prevents attending to future positions.

Create a test where you:
1. Use your `create_causal_mask` function
2. Call `scaled_dot_product_attention` with the mask
3. Verify that future positions have zero attention weight

Set `causal_attention_weights` to the attention weights from applying causal masking to the provided query, key, and value tensors.

In [ ]:
# Setup: create test tensors
torch.manual_seed(123)
test_batch_size, test_seq_len, test_embed_dim = 1, 5, 4
test_query = torch.randn(test_batch_size, test_seq_len, test_embed_dim)
test_key = torch.randn(test_batch_size, test_seq_len, test_embed_dim)
test_value = torch.randn(test_batch_size, test_seq_len, test_embed_dim)

# BEGIN SOLUTION
# Create causal mask and apply attention
causal_mask = create_causal_mask(test_seq_len)
_, causal_attention_weights = scaled_dot_product_attention(
    test_query, test_key, test_value, mask=causal_mask
)
# END SOLUTION

In [ ]:
# Test assertions
assert causal_attention_weights.shape == (
    test_batch_size,
    test_seq_len,
    test_seq_len,
), "Attention weights have incorrect shape"

# Check that upper triangle (future positions) has zero attention
upper_triangle = torch.triu(causal_attention_weights[0], diagonal=1)
assert torch.allclose(
    upper_triangle, torch.zeros_like(upper_triangle), atol=1e-6
), "Future positions should have zero attention weight"

# Check rows still sum to 1
row_sums = causal_attention_weights.sum(dim=-1)
assert torch.allclose(
    row_sums, torch.ones_like(row_sums), atol=1e-5
), "Attention weights should still sum to 1"
print("All tests passed!")

# BEGIN HIDDEN TESTS
# First position should only attend to itself
assert torch.allclose(
    causal_attention_weights[0, 0, 0], torch.tensor(1.0), atol=1e-5
), "Position 0 should have attention weight 1.0 on itself"

# Last position should have non-zero weights for all positions
last_row = causal_attention_weights[0, -1, :]
assert (last_row > 0).all(), "Last position should attend to all positions"
# END HIDDEN TESTS

## Background: Positional Encoding

Unlike RNNs, transformers process all positions in parallel and have no inherent notion of order. **Positional encodings** are added to the input embeddings to give the model information about the position of each token.

The original transformer paper uses sinusoidal positional encodings:

$$PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d}}\right)$$
$$PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d}}\right)$$

where $pos$ is the position and $i$ is the dimension index. This creates a unique encoding for each position that the model can learn to use.

### Problem 4: Implement Sinusoidal Positional Encoding

Implement `create_positional_encoding` that generates sinusoidal positional encodings.

**Parameters:**
- `max_seq_len`: Maximum sequence length
- `embed_dim`: Embedding dimension (must be even)

**Returns:**
- Tensor of shape `(max_seq_len, embed_dim)` containing the positional encodings

**Algorithm:**
1. Create position indices: `pos = [0, 1, 2, ..., max_seq_len-1]`
2. Create dimension indices: `i = [0, 1, 2, ..., embed_dim/2-1]`
3. Compute the divisor term: `div_term = 10000^(2i/embed_dim)`
4. Apply sine to even indices: `PE[:, 0::2] = sin(pos / div_term)`
5. Apply cosine to odd indices: `PE[:, 1::2] = cos(pos / div_term)`

**Hint:** Use `torch.arange` and broadcasting. The divisor can be computed as `torch.exp(torch.arange(0, embed_dim, 2) * -(math.log(10000.0) / embed_dim))`.

In [ ]:
def create_positional_encoding(max_seq_len, embed_dim):
    """
    Create sinusoidal positional encodings.

    Parameters:
    - max_seq_len: Maximum sequence length
    - embed_dim: Embedding dimension (must be even)

    Returns:
    - pe: Tensor of shape (max_seq_len, embed_dim)
    """
    # BEGIN SOLUTION
    # Create position indices (max_seq_len, 1)
    position = torch.arange(0, max_seq_len).unsqueeze(1).float()

    # Compute the divisor term for each dimension pair
    # div_term[i] = 1 / 10000^(2i/embed_dim) = exp(-2i * log(10000) / embed_dim)
    div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * -(math.log(10000.0) / embed_dim))

    # Initialize positional encoding matrix
    pe = torch.zeros(max_seq_len, embed_dim)

    # Apply sine to even indices
    pe[:, 0::2] = torch.sin(position * div_term)

    # Apply cosine to odd indices
    pe[:, 1::2] = torch.cos(position * div_term)

    return pe
    # END SOLUTION

In [ ]:
# Test assertions
pe = create_positional_encoding(100, 64)

assert pe.shape == (100, 64), f"Shape should be (100, 64), got {pe.shape}"

# Position 0 should have sin(0)=0 for even dims and cos(0)=1 for odd dims
assert torch.allclose(
    pe[0, 0::2], torch.zeros(32), atol=1e-5
), "Position 0, even dims should be sin(0)=0"
assert torch.allclose(
    pe[0, 1::2], torch.ones(32), atol=1e-5
), "Position 0, odd dims should be cos(0)=1"

# Values should be bounded between -1 and 1
assert pe.min() >= -1.0 and pe.max() <= 1.0, "Positional encodings should be in [-1, 1]"
print("All tests passed!")

# BEGIN HIDDEN TESTS
# Test with smaller dimensions
pe_small = create_positional_encoding(10, 4)
assert pe_small.shape == (10, 4), "Shape mismatch for small PE"

# Check that different positions have different encodings
assert not torch.allclose(
    pe[0], pe[1], atol=1e-3
), "Different positions should have different encodings"

# Verify the pattern: low-frequency dims (higher indices) change more slowly
# Compare rate of change between position 0->1 for dim 0 vs dim 62
diff_low_dim = abs(pe[1, 0] - pe[0, 0])
diff_high_dim = abs(pe[1, 62] - pe[0, 62])
assert diff_low_dim > diff_high_dim, "Lower dimension indices should change faster"
# END HIDDEN TESTS

## Background: Complete Transformer Model

Now let's put it all together. A transformer-based language model typically consists of:

1. **Token Embedding**: Converts token indices to dense vectors
2. **Positional Encoding**: Adds position information
3. **Transformer Layers**: Self-attention + feedforward networks
4. **Output Projection**: Maps back to vocabulary size

PyTorch provides `nn.TransformerEncoder` which handles the transformer layers. We just need to handle the embeddings and output.

### Problem 5: Complete the Transformer Language Model

Complete the `TransformerLM` class by implementing the `forward` method. The model should:

1. Convert input token indices to embeddings
2. Add positional encodings (already stored in `self.positional_encoding`)
3. Create a causal mask using your `create_causal_mask` function
4. Pass through the transformer encoder with the causal mask
5. Project to vocabulary size and return log probabilities

**Parameters:**
- `x`: Input tensor of shape `(batch_size, seq_len)` containing token indices

**Returns:**
- Log probabilities of shape `(batch_size, seq_len, vocab_size)`

**Important:** PyTorch's `TransformerEncoder` uses a different mask convention. It expects a float mask where masked positions are `-inf` and unmasked positions are `0`. The mask shape should be `(seq_len, seq_len)`. Use `torch.where` to convert your boolean mask.

In [ ]:
class TransformerLM(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, hidden_dim, num_layers, max_seq_len):
        """
        Initialize the Transformer Language Model.

        Parameters:
        - vocab_size: Number of tokens in vocabulary
        - embed_dim: Embedding dimension
        - num_heads: Number of attention heads
        - hidden_dim: Hidden dimension in feedforward layers
        - num_layers: Number of transformer layers
        - max_seq_len: Maximum sequence length
        """
        super().__init__()

        self.embed_dim = embed_dim

        # Token embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_dim)

        # Store positional encoding as a buffer (not a parameter)
        pe = create_positional_encoding(max_seq_len, embed_dim)
        self.register_buffer("positional_encoding", pe)

        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            dim_feedforward=hidden_dim,
            batch_first=True,
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # Output projection
        self.output_layer = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        """
        Forward pass of the model.

        Parameters:
        - x: Input tensor of shape (batch_size, seq_len) containing token indices

        Returns:
        - Log probabilities of shape (batch_size, seq_len, vocab_size)
        """
        # BEGIN SOLUTION
        seq_len = x.size(1)

        # Step 1: Get token embeddings
        token_embeddings = self.embedding(x)  # (batch, seq_len, embed_dim)

        # Step 2: Add positional encodings
        # Only use the first seq_len positions
        x = token_embeddings + self.positional_encoding[:seq_len, :]

        # Step 3: Create causal mask
        # PyTorch expects float mask: 0 for attend, -inf for mask
        causal_mask = create_causal_mask(seq_len)
        attn_mask = torch.where(causal_mask, torch.tensor(float("-inf")), torch.tensor(0.0)).to(
            x.device
        )

        # Step 4: Pass through transformer
        x = self.transformer(x, mask=attn_mask)

        # Step 5: Project to vocabulary and apply log softmax
        logits = self.output_layer(x)
        return functional.log_softmax(logits, dim=-1)
        # END SOLUTION

In [ ]:
# Test assertions
torch.manual_seed(42)

# Create a small model for testing
test_vocab_size = 100
test_embed_dim = 32
test_num_heads = 4
test_hidden_dim = 64
test_num_layers = 2
test_max_seq_len = 20

model = TransformerLM(
    vocab_size=test_vocab_size,
    embed_dim=test_embed_dim,
    num_heads=test_num_heads,
    hidden_dim=test_hidden_dim,
    num_layers=test_num_layers,
    max_seq_len=test_max_seq_len,
)

# Test with a batch of sequences
test_batch = torch.randint(0, test_vocab_size, (4, 10))  # 4 sequences of length 10
output = model(test_batch)

assert output.shape == (
    4,
    10,
    test_vocab_size,
), f"Output shape should be (4, 10, {test_vocab_size}), got {output.shape}"

# Log probabilities should sum to 0 (in log space, probabilities sum to 1)
prob_sums = output.exp().sum(dim=-1)
assert torch.allclose(
    prob_sums, torch.ones_like(prob_sums), atol=1e-4
), "Probabilities should sum to 1"
print("All tests passed!")

# BEGIN HIDDEN TESTS
# Test with different sequence lengths
test_batch_short = torch.randint(0, test_vocab_size, (2, 5))
output_short = model(test_batch_short)
assert output_short.shape == (
    2,
    5,
    test_vocab_size,
), "Should handle variable sequence lengths"

# Test that model is deterministic with same input
model.eval()
with torch.no_grad():
    out1 = model(test_batch)
    out2 = model(test_batch)
assert torch.allclose(out1, out2), "Model should be deterministic in eval mode"

# Test that output values are in valid log probability range
assert (output <= 0).all(), "Log probabilities should be <= 0"
# END HIDDEN TESTS

### Problem 6: Compute Perplexity

**Perplexity** is a common metric for evaluating language models. It measures how "surprised" the model is by the test data. Lower perplexity means the model assigns higher probability to the actual next tokens.

Perplexity is defined as:

$$\text{Perplexity} = \exp\left(-\frac{1}{N}\sum_{i=1}^{N} \log P(w_i | w_1, ..., w_{i-1})\right)$$

which is the exponential of the average negative log-likelihood (cross-entropy loss).

Implement `compute_perplexity` that calculates the perplexity of a model on a batch of sequences.

**Parameters:**
- `model`: A TransformerLM model
- `sequences`: Tensor of shape `(batch_size, seq_len)` containing token indices

**Returns:**
- A scalar tensor containing the perplexity

**Hint:** The model outputs log probabilities for predicting the next token at each position. For a sequence `[w0, w1, w2, w3]`, position 0's output predicts `w1`, position 1's output predicts `w2`, etc. Use `sequences[:, 1:]` as targets and `log_probs[:, :-1, :]` as predictions.

In [ ]:
def compute_perplexity(model, sequences):
    """
    Compute the perplexity of a language model on a batch of sequences.

    Parameters:
    - model: A TransformerLM model
    - sequences: Tensor of shape (batch_size, seq_len)

    Returns:
    - perplexity: Scalar tensor
    """
    # BEGIN SOLUTION
    model.eval()
    with torch.no_grad():
        # Get log probabilities from the model
        log_probs = model(sequences)  # (batch, seq_len, vocab_size)

        # Shift: use positions 0 to seq_len-2 to predict positions 1 to seq_len-1
        # Predictions: log_probs[:, :-1, :] -> (batch, seq_len-1, vocab_size)
        # Targets: sequences[:, 1:] -> (batch, seq_len-1)
        predictions = log_probs[:, :-1, :]
        targets = sequences[:, 1:]

        # Gather the log probabilities of the actual next tokens
        # targets.unsqueeze(-1) -> (batch, seq_len-1, 1)
        target_log_probs = predictions.gather(dim=-1, index=targets.unsqueeze(-1)).squeeze(-1)

        # Compute average negative log likelihood
        avg_nll = -target_log_probs.mean()

        # Perplexity is exp of average NLL
        return torch.exp(avg_nll)
    # END SOLUTION

In [ ]:
# Test assertions
torch.manual_seed(42)

# Use the model from Problem 5
test_sequences = torch.randint(0, test_vocab_size, (8, 15))
perplexity = compute_perplexity(model, test_sequences)

assert perplexity.dim() == 0, "Perplexity should be a scalar"
assert perplexity > 0, "Perplexity should be positive"

# For a random untrained model, perplexity should be around vocab_size
assert (
    perplexity > 50 and perplexity < 200
), f"For untrained model, perplexity ~{test_vocab_size}, got {perplexity:.1f}"
print(f"Perplexity: {perplexity:.2f}")
print("All tests passed!")

# BEGIN HIDDEN TESTS
# Perplexity should be consistent
perplexity2 = compute_perplexity(model, test_sequences)
assert torch.allclose(perplexity, perplexity2), "Perplexity should be deterministic"

# Test with a single sequence
single_seq = torch.randint(0, test_vocab_size, (1, 10))
single_perp = compute_perplexity(model, single_seq)
assert single_perp.dim() == 0, "Should work with single sequence"
assert single_perp > 0, "Should be positive for single sequence"
# END HIDDEN TESTS

## Summary

In this assignment, you implemented:

1. **Scaled dot-product attention** - the core attention mechanism
2. **Causal masking** - preventing attention to future positions
3. **Sinusoidal positional encoding** - adding position information
4. **A complete transformer language model** - combining all components
5. **Perplexity computation** - evaluating language model quality

These are the fundamental building blocks of modern language models like GPT. The actual models used in practice have many more layers, larger dimensions, and are trained on massive datasets, but the core mechanisms are the same as what you implemented here.